# 【問題1】ブレンディング
ブレンディングを実装し、単一モデルより精度があがる例を最低3つ示してください。  
精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを示します。  
ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。  
最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。  
手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）  
ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）  
入力データの前処理の仕方（例：標準化、対数変換、PCAなど）  
重要なのはそれぞれのモデルが大きく異なることです。必ずしも単一モデルの精度が高い必要はありません。  
回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。  
補足  
分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。  
sklearn.ensemble.VotingClassifier — scikit-learn 0.20.0 documentation  
考察  
どういった組み合わせが良いか、どのようにすると多様なモデルが作れるかを考えてみましょう。  

## 前処理

###### データの用意と確認

In [206]:
import pandas as pd
import numpy as np

In [207]:
# CSVデータを読み込み
df = pd.read_csv('house_price_dataset_train.csv',header=0)
#データ形状を表示
print('df shape: (%i,%i)' %df.shape)
df.head()

df shape: (1460,81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


###### Xとyのデータセットを作成

In [208]:
X  = df.iloc[:, 1:-1]            # 1列目から最終列以前を特徴量
ID = X.iloc[:, [0]]             # 第0列はPK（Loan_ID）なのでIDとしてセット
#X  = X.drop('カラム名', axis=1)  # 特徴ベクトルから削除したいカラム名を指定
y  = df.iloc[:, [-1]]           # 最終列を正解データ

###### データの統計情報と欠損値の情報を確認

In [209]:
###### 欠損値の数、欠損値の割合を算出してデータフレームに共有
X_missingno = pd.DataFrame(index=X.columns)
X_missingno['missingno_counts'] = X.isnull().sum() #欠損値の数を挿入
X_missingno['missingno_rates'] = X.isnull().sum()/len(X)#欠損値の割合を挿入
X_missingno['data_type'] = X.dtypes #欠損値の数を挿入
X_missingno['unique_elements_counts'] = X.nunique() #要素の個数をカウント
X_missingno['unique_elements_rates'] = X.nunique()/len(X) #要素の個数が全体に占める割合（離散値の判定）
#X_missingno[X_missingno['missingno_counts']>0] #欠損値のある特徴量のみ表示

######  統計情報と欠損情報を表示
pd.concat([X.describe(),X_missingno.T],sort=False)

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1201,1460,1460,1460,1460,1460,1452,1460,1460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,56.8973,70.05,10516.8,6.09932,5.57534,1971.27,1984.87,103.685,443.64,46.5493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,42.3006,24.2848,9981.26,1.383,1.1128,30.2029,20.6454,181.066,456.098,161.319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,20,21,1300,1,1,1872,1950,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,20,59,7553.5,5,5,1954,1967,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,50,69,9478.5,6,5,1973,1994,0,383.5,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,70,80,11601.5,7,6,2000,2004,166,712.25,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,190,313,215245,10,9,2010,2010,1600,5644,1474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
missingno_counts,0,259,0,0,0,0,0,8,0,0,...,81,81,81,81,0,1453,1179,1406,0,0
missingno_rates,0,0.177397,0,0,0,0,0,0.00547945,0,0,...,0.0554795,0.0554795,0.0554795,0.0554795,0,0.995205,0.807534,0.963014,0,0


##### object型のカラムの抽出とohe-hotエンコーディング

In [210]:
#object型のみ抽出
#データオブジェクトリストの作成
X_object_list = X.select_dtypes(include=object).columns

In [211]:
#one-hot encording
#データタイプがオブジェクトの特徴量をone-hot encodingする
ohe_columns = X_object_list
X_ohe = pd.get_dummies(X,
                       dummy_na=True, # NANもone-hotエンコーディングする
                       columns=ohe_columns)
X_ohe.head() #念のため表示して特徴量数を確認

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,1,0,0,0,0,0,1,0,0
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,1,0,0,0,0,0,1,0,0
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,1,0,0,0,0,0,1,0,0
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,1,0,1,0,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,1,0,0,0,0,0,1,0,0


###### 欠損値の補完

In [212]:
from sklearn.preprocessing import Imputer

# 欠損値NaNを平均値(mean)で置換
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_ohe)

# 学習済みImputerを適用しX_newの欠損値を置換
X_ohe_columns = X_ohe.columns.values
X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)

# 結果表示
X_ohe.head()

/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_Oth,SaleType_WD,SaleType_nan,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,SaleCondition_nan
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


###### RFE特徴量選択を実施

In [213]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

selector = RFE(RandomForestClassifier(random_state=1),
               n_features_to_select=30,
               step=.05)
selector.fit(X_ohe,
             y.as_matrix().ravel())

X_fin = X_ohe.loc[:, X_ohe_columns[selector.support_]]
print('X_fin shape:(%i,%i)' % X_fin.shape)
X_fin.head()

/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tera/.pyenv/ve

X_fin shape:(1460,30)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,MoSold,YrSold,LotShape_IR1,LotConfig_Inside,BsmtExposure_No,GarageFinish_RFn
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,548.0,0.0,61.0,0.0,2.0,2008.0,0.0,1.0,1.0,1.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,460.0,298.0,0.0,0.0,5.0,2007.0,0.0,0.0,0.0,1.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,608.0,0.0,42.0,0.0,9.0,2008.0,1.0,1.0,0.0,1.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,642.0,0.0,35.0,272.0,2.0,2006.0,1.0,0.0,1.0,0.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,836.0,192.0,84.0,0.0,12.0,2008.0,1.0,0.0,0.0,1.0


###### データの標準化を行う

In [256]:
# import libraries
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score


#Xを標準化
#yは重要度を求めるなら標準化 y_std = scl.fit_transform(y)
scl_X = StandardScaler()
#scl_y = StandardScaler()
X_std = scl_X.fit_transform(X_fin)
#y_std = scl_y.fit_transform(y)

# 交差検証(holdout)
X_train,X_test,y_train,y_test = train_test_split(X_std,
                                                 y,
                                                 test_size=0.30,
                                                 random_state=0)

# データフレームを配列に整形 #やらなくてもできるが警告が出る
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

## モデルの作成→フィッティング→スコア検証

In [265]:
#各モデル作成
ols = LinearRegression()
ridge = Ridge(random_state=0)
rf = RandomForestRegressor(random_state=0)
gbr = GradientBoostingRegressor(random_state=0)
lgb = LGBMRegressor(random_state=0)
mlp = MLPRegressor(random_state=0)
lgb_new = LGBMRegressor(learning_rate=0.2,random_state=0)

# モデルを作成しフィットさせる
ols.fit(X_train,y_train)
ridge.fit(X_train,y_train)
rf.fit(X_train,y_train)

# 勾配ブースティングは時間短縮のため、一つに選んでもよい
gbr.fit(X_train,y_train) # 「#」を先頭に追加することでその計算は行わないようにする
lgb.fit(X_train,y_train) # 「#」を先頭に追加することでその計算は行わないようにする
lgb_new.fit(X_train,y_train) # 「#」を先頭に追加することでその計算は行わないようにする
#mlp.fit(X_train,y_train) # 「#」を先頭に追加することでその計算は行わないようにする

# 回帰分析なのでR2スコアで精度を比較
print('OLS: %.6f' % r2_score(y_test,ols.predict(X_test)))
print('Ridge: %.6f' % r2_score(y_test,ridge.predict(X_test)))
print('RF: %.6f' % r2_score(y_test,rf.predict(X_test)))
print('GBR: %.6f' % r2_score(y_test,gbr.predict(X_test))) # 「#」を先頭に追加することでその計算は行わないようにする
print('LGBM: %.6f' % r2_score(y_test,lgb.predict(X_test))) # 「#」を先頭に追加することでその計算は行わないようにする
print('LGBM NEW: %.6f' % r2_score(y_test,lgb_new.predict(X_test))) # 「#」を先頭に追加することでその計算は行わないようにする
#print('MLP: %.6f' % r2_score(y_test,mlp.predict(X_test))) # 「#」を先頭に追加することでその計算は行わないようにする

/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


OLS: 0.725757
Ridge: 0.725885
RF: 0.838454
GBR: 0.848453
LGBM: 0.848260
LGBM NEW: 0.852751


## ブレンディングを実施

ランダムフォレストとLightGBMのブレンディング

In [263]:
y_pred_blended_1 = np.mean((rf.predict(X_test),lgb.predict(X_test)),axis=0)
print('BLENDED MODEL 1: %.6f' % r2_score(y_test,y_pred_blended_1)) # ブレンドされたモデルの検証

BLENDED MODEL 1: 0.851397


ランダムフォレストとLightGBMと勾配ブースティングのブレンディング

In [262]:
y_pred_blended_2 = np.mean((rf.predict(X_test),lgb.predict(X_test),gbr.predict(X_test)),axis=0)
print('BLENDED MODEL 2: %.6f' % r2_score(y_test,y_pred_blended_2)) # ブレンドされたモデルの検証

BLENDED MODEL 2: 0.855257


学習率がデフォルト0.1のLightGBMと学習率が0.2のLightGBMのブレンディング

In [267]:
y_pred_blended_3 = np.mean((lgb.predict(X_test),lgb_new.predict(X_test)),axis=0)
print('BLENDED MODEL 3: %.6f' % r2_score(y_test,y_pred_blended_3)) # ブレンドされたモデルの検証

BLENDED MODEL 3: 0.853556


<font color='red'>もともとスコアの高いモデルの組み合わせによって大きくモデルを進化させることができる</font>

# 【問題2】バギング
バギングを実装し、単一モデルより精度があがる例を最低1つ示してください。
バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ブートストラップサンプル）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。
sklearn.model_selection.train_test_split — scikit-learn 0.20.0 documentation
scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。
推定結果の平均をとる部分はブースティングと同様の実装になります。

###### ランダムにインデックスを作成

In [268]:
import random

#ランダムなインデックスを作成
random_index_1 = random.sample(range(0,len(X_train)),1000)
random_index_2 = random.sample(range(0,len(X_train)),1000)
random_index_3 = random.sample(range(0,len(X_train)),1000)
random_index_4 = random.sample(range(0,len(X_train)),1000)
random_index_5 = random.sample(range(0,len(X_train)),1000)

In [269]:
#ランダムなインデックスのX_trainデータを抽出
X_train_1 = X_train[random_index_1]
X_train_2 = X_train[random_index_2]
X_train_3 = X_train[random_index_3]
X_train_4 = X_train[random_index_4]
X_train_5 = X_train[random_index_5]

y_train_1 = y_train[random_index_1]
y_train_2 = y_train[random_index_2]
y_train_3 = y_train[random_index_3]
y_train_4 = y_train[random_index_4]
y_train_5 = y_train[random_index_5]

In [270]:
#5つのlgbモデルを作成し、学習を進める
lgb_bagging_1 = LGBMRegressor(random_state=0)
lgb_bagging_2 = LGBMRegressor(random_state=0)
lgb_bagging_3 = LGBMRegressor(random_state=0)
lgb_bagging_4 = LGBMRegressor(random_state=0)
lgb_bagging_5 = LGBMRegressor(random_state=0)

lgb_bagging_1.fit(X_train_1,y_train_1)
lgb_bagging_2.fit(X_train_2,y_train_2)
lgb_bagging_3.fit(X_train_3,y_train_3)
lgb_bagging_4.fit(X_train_4,y_train_4)
lgb_bagging_5.fit(X_train_5,y_train_5)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=0, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [271]:
#バギングした予測値を算出
y_pred_gbagging = (lgb_bagging_1.predict(X_test) + lgb_bagging_2.predict(X_test) + lgb_bagging_3.predict(X_test) \
                          + lgb_bagging_4.predict(X_test) + lgb_bagging_5.predict(X_test)) / 5

# バギングしたモデルの検証
print('BAGGING LightGBM MODEL: %.6f' % r2_score(y_test,y_pred_gbagging))
print('Non BAGGING LightGBM MODEL: %.6f' % 0.848260)

BAGGING LightGBM MODEL: 0.849139
Non BAGGING LightGBM MODEL: 0.848260


# 【問題3】スタッキング
スタッキングを実装し、単一モデルより精度があがる例を最低1つ示してください。  
スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください

In [272]:
#学習用データとテストデータに分割
X_train,X_test,y_train,y_test = train_test_split(X_std,
                                                 y,
                                                 test_size=0.30,
                                                 random_state=0)

# データフレームを配列に整形 #やらなくてもできるが警告が出る
y_train = y_train.values.ravel()
y_test = y_test.values.ravel().reshape(-1,)

###### 学習-ステージ0

In [273]:
train_index_0 = list(range(0,len(X_train)))
split_n_0 = 3 #分割数を指定
index_list_0 = [train_index_0[i::split_n_0] for i in range(split_n_0)] #リストの分割を実施

In [287]:
index_list_0

[[0,
  3,
  6,
  9,
  12,
  15,
  18,
  21,
  24,
  27,
  30,
  33,
  36,
  39,
  42,
  45,
  48,
  51,
  54,
  57,
  60,
  63,
  66,
  69,
  72,
  75,
  78,
  81,
  84,
  87,
  90,
  93,
  96,
  99,
  102,
  105,
  108,
  111,
  114,
  117,
  120,
  123,
  126,
  129,
  132,
  135,
  138,
  141,
  144,
  147,
  150,
  153,
  156,
  159,
  162,
  165,
  168,
  171,
  174,
  177,
  180,
  183,
  186,
  189,
  192,
  195,
  198,
  201,
  204,
  207,
  210,
  213,
  216,
  219,
  222,
  225,
  228,
  231,
  234,
  237,
  240,
  243,
  246,
  249,
  252,
  255,
  258,
  261,
  264,
  267,
  270,
  273,
  276,
  279,
  282,
  285,
  288,
  291,
  294,
  297,
  300,
  303,
  306,
  309,
  312,
  315,
  318,
  321,
  324,
  327,
  330,
  333,
  336,
  339,
  342,
  345,
  348,
  351,
  354,
  357,
  360,
  363,
  366,
  369,
  372,
  375,
  378,
  381,
  384,
  387,
  390,
  393,
  396,
  399,
  402,
  405,
  408,
  411,
  414,
  417,
  420,
  423,
  426,
  429,
  432,
  435,
  438,
  441,
  

In [274]:
#学習用Xデータを分割
X_stacking_00 = X_train[index_list_0[0]] 
X_stacking_01 = X_train[index_list_0[1]]
X_stacking_02 = X_train[index_list_0[2]]

#学習用yデータを分割
y_stacking_00 = np.array(y_train)[index_list_0[0]]
y_stacking_01 = np.array(y_train)[index_list_0[1]]
y_stacking_02 = np.array(y_train)[index_list_0[2]]

In [275]:
X_stacking_00_mix = np.concatenate([X_stacking_01,X_stacking_02],axis=0)
X_stacking_01_mix = np.concatenate([X_stacking_02,X_stacking_00],axis=0)
X_stacking_02_mix = np.concatenate([X_stacking_00,X_stacking_01],axis=0)

y_stacking_00_mix = np.concatenate([y_stacking_01,y_stacking_02],axis=0)
y_stacking_01_mix = np.concatenate([y_stacking_02,y_stacking_00],axis=0)
y_stacking_02_mix = np.concatenate([y_stacking_00,y_stacking_01],axis=0)

###### モデルをセットして学習

In [276]:
lgb_stacking_00 = LGBMRegressor(random_state=0)
lgb_stacking_01 = LGBMRegressor(random_state=0)
lgb_stacking_02 = LGBMRegressor(random_state=0)
rf_stacking_00 = RandomForestRegressor(random_state=0)
rf_stacking_01 = RandomForestRegressor(random_state=0)
rf_stacking_02 = RandomForestRegressor(random_state=0)

lgb_stacking_00.fit(X_stacking_00_mix,y_stacking_00_mix)
lgb_stacking_01.fit(X_stacking_01_mix,y_stacking_01_mix)
lgb_stacking_02.fit(X_stacking_02_mix,y_stacking_02_mix)
rf_stacking_00.fit(X_stacking_00_mix,y_stacking_00_mix)
rf_stacking_01.fit(X_stacking_01_mix,y_stacking_01_mix)
rf_stacking_02.fit(X_stacking_02_mix,y_stacking_02_mix)

/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [277]:
lgb_blend_data_0 = lgb_stacking_00.predict(X_stacking_00).reshape(-1,1)
lgb_blend_data_1 = lgb_stacking_01.predict(X_stacking_01).reshape(-1,1)
lgb_blend_data_2 = lgb_stacking_02.predict(X_stacking_02).reshape(-1,1)

rf_blend_data_0 = rf_stacking_00.predict(X_stacking_00).reshape(-1,1)
rf_blend_data_1 = rf_stacking_01.predict(X_stacking_01).reshape(-1,1)
rf_blend_data_2 = rf_stacking_02.predict(X_stacking_02).reshape(-1,1)

lgb_blend_data_X_0 = np.concatenate([lgb_blend_data_0,lgb_blend_data_1,lgb_blend_data_2],axis=0) # lgbで予測したデータを結合する
rf_blend_data_X_0 = np.concatenate([rf_blend_data_0,rf_blend_data_1,rf_blend_data_2],axis=0) # rfで予測したデータを結合する
y_train_shuffle = np.concatenate([y_stacking_00,y_stacking_01,y_stacking_02],axis=0) # y_trainのデータをシャッフルした順に結合する

blend_data_X_0 = np.concatenate([lgb_blend_data_X_0,rf_blend_data_X_0],axis=1) # 全てのデータを結合する

###### 学習-ステージ１（最終ステージ）

In [278]:
#リッジ回帰のインスタンス作成と学習
ridge_stacking_10 = Ridge(random_state=0)
ridge_stacking_10.fit(blend_data_X_0,y_train_shuffle)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=0, solver='auto', tol=0.001)

###### 推定-ステージ0

In [279]:
#ステージ0で作成したモデルにテストデータを入れて予測を実施。
y_lgb_pred_00 = lgb_stacking_00.predict(X_test).reshape(-1,1)
y_lgb_pred_01 = lgb_stacking_01.predict(X_test).reshape(-1,1)
y_lgb_pred_02 = lgb_stacking_02.predict(X_test).reshape(-1,1)
y_rf_pred_00 = rf_stacking_00.predict(X_test).reshape(-1,1)
y_rf_pred_01 = rf_stacking_01.predict(X_test).reshape(-1,1)
y_rf_pred_02 = rf_stacking_02.predict(X_test).reshape(-1,1)

In [280]:
#LightGBMの予測が３つ完成したが、予測値の平均をとって（ブレンディング）、予測とする。
y_lgb_pred_00_01_02 = np.mean([y_lgb_pred_00,y_lgb_pred_01,y_lgb_pred_02],axis=0)

#RandomForestの予測が３つ完成したが、予測値の平均をとって（ブレンディング）、予測とする。
y_rf_pred_00_01_02 = np.mean([y_rf_pred_00,y_rf_pred_01,y_rf_pred_02],axis=0)

#３インスタンスのブレンディングで作成したモデルの予測を２モデル分結合してデータを作る。
y_lgb_rf_pred_00_01_02 = np.concatenate([y_lgb_pred_00_01_02 ,y_rf_pred_00_01_02 ],axis=1)

###### 推定-ステージ1

In [281]:
#学習時最終ステージで作成したモデルにステージ０で作成したデータを流し込む
y_final_pred_1_ridge = ridge_stacking_10.predict(y_lgb_rf_pred_00_01_02)

###### スタッキングにおけるスコアリング

In [282]:
# スタッキングしたモデルの検証
print('Stacking LightGBM+RandomForest→Ridge MODEL: %.6f' % r2_score(y_test,y_final_pred_1_ridge))
print('Non Stacking LightGBM MODEL: %.6f' % 0.848260)

Stacking LightGBM+RandomForest→Ridge MODEL: 0.849450
Non Stacking LightGBM MODEL: 0.848260


# 【問題4】（アドバンス課題）Microsoft Malware Prediction
時間的に余裕がある場合は、KaggleのMalwareコンペを題材に、アンサンブル学習を実際に利用してみましょう。   
EDAからはじめ、推定値の提出まで挑戦してください。  
Microsoft Malware Prediction | Kaggleです。  
注意点  
非常に大きなデータセットなため、扱いに工夫が必要です

In [ ]:
<指針>  
非常に大きなデータではあるので、

In [284]:
df_advanced = pd.read_csv('microsoft_malware prediction_train.csv',header=0)

/Users/tera/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [292]:
print(df_advanced.shape)
df_advanced.drop()

(8921483, 83)


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,0000028988387b115f69f31a3bf04f09,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1735.0,0,7.0,0,NaN,53447.0,...,36144.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
1,000007535c3f730efa9ea0b7ef1bd645,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,53447.0,...,57858.0,0,NaN,0.0,0,0,0.0,0.0,8.0,0
2,000007905a28d863f6d0d597892cd692,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1341.0,0,7.0,0,NaN,53447.0,...,52682.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
3,00000b11598a75ea8ba1beea8459149f,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1527.0,0,7.0,0,NaN,53447.0,...,20050.0,0,NaN,0.0,0,0,0.0,0.0,3.0,1
4,000014a5f00daa18e76b81417eeb99fc,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1379.0,0,7.0,0,NaN,53447.0,...,19844.0,0,0.0,0.0,0,0,0.0,0.0,1.0,1
5,000016191b897145d069102325cab760,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1094.0,0,7.0,0,NaN,53447.0,...,51039.0,0,0.0,0.0,0,0,0.0,0.0,15.0,1
6,0000161e8abf8d8b89c5ab8787fd712b,win8defender,1.1.15100.1,4.18.1807.18075,1.273.845.0,0,7.0,0,NaN,43927.0,...,63175.0,1,NaN,0.0,0,0,0.0,0.0,10.0,1
7,000019515bc8f95851aff6de873405e8,win8defender,1.1.15100.1,4.18.1807.18075,1.273.1393.0,0,7.0,0,NaN,53447.0,...,63122.0,0,0.0,0.0,0,0,0.0,0.0,15.0,0
8,00001a027a0ab970c408182df8484fce,win8defender,1.1.15200.1,4.18.1807.18075,1.275.988.0,0,7.0,0,NaN,53447.0,...,15510.0,0,0.0,0.0,0,0,0.0,0.0,15.0,0
9,00001a18d69bb60bda9779408dcf02ac,win8defender,1.1.15100.1,4.18.1807.18075,1.273.973.0,0,7.0,0,NaN,46413.0,...,63555.0,1,0.0,0.0,0,0,0.0,1.0,8.0,1


In [293]:
X_advance  = df_advanced.iloc[:, 1:-1]            # 1列目から最終列以前を特徴量
ID = X_advance.iloc[:, [0]]             # 第0列はPK（Loan_ID）なのでIDとしてセット
#X  = X.drop('カラム名', axis=1)  # 特徴ベクトルから削除したいカラム名を指定
y_advance  = df_advanced.iloc[:, [-1]]           # 最終列を正解データ

In [298]:
y_advance.describe()

,HasDetections
count,8.921483e+06
mean,4.997927e-01
std,5.000000e-01
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,1.000000e+00
max,1.000000e+00


In [294]:
###### 欠損値の数、欠損値の割合を算出してデータフレームに共有
X_missingno = pd.DataFrame(index=X_advance.columns)
X_missingno['missingno_counts'] = X_advance.isnull().sum() #欠損値の数を挿入
X_missingno['missingno_rates'] = X_advance.isnull().sum()/len(X_advance)#欠損値の割合を挿入
X_missingno['data_type'] = X_advance.dtypes #欠損値の数を挿入
X_missingno['unique_elements_counts'] = X_advance.nunique() #要素の個数をカウント
X_missingno['unique_elements_rates'] = X_advance.nunique()/len(X_advance) #要素の個数が全体に占める割合（離散値の判定）
#X_missingno[X_missingno['missingno_counts']>0] #欠損値のある特徴量のみ表示

######  統計情報と欠損情報を表示
pd.concat([X_advance.describe(),X_missingno.T],sort=False)

,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,...,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSWUAutoUpdateOptionsName,Census_GenuineStateName,Census_ActivationChannel,Census_FlightRing
count,8.92148e+06,8.88916e+06,8.92148e+06,433438,8.88526e+06,8.88526e+06,8.88526e+06,8.92148e+06,8.92148e+06,8.59607e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,7.50996e-06,6.84533,0.0173338,1658.36,47840,1.32678,1.02097,0.987971,108.049,81266.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.00274042,1.02606,0.130512,998.96,14032.4,0.522927,0.167554,0.109015,63.0471,48923.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0,0,0,1,3,0,0,0,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0,7,0,788,49480,1,1,1,51,36825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0,7,0,1632,53447,1,1,1,97,82373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,0,7,0,2373,53447,2,1,1,162,123700,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1,35,1,3213,70507,7,5,1,222,167962,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
missingno_counts,0,32318,0,8488045,36221,36221,36221,0,0,325409,...,0,0,0,0,0,0,0,0,0,0
missingno_rates,0,0.00362249,0,0.951416,0.00405998,0.00405998,0.00405998,0,0,0.0364748,...,0,0,0,0,0,0,0,0,0,0


yのラベルは均衡データになっている  
だが、データが重たくなる傾向があり、優れたモデルの生成方法を検討するに当たって１０万サンプルにデータ数を絞ることにする。

In [303]:
len(X_advance)

8921483

In [304]:
len(sample_index)

100000

In [326]:
import random

sample_index = random.sample(range(0,len(X_advance)),100000)
X_advance.iloc[sample_index,:]
y_advance.iloc[sample_index,:]

,HasDetections
5162728,0
2542546,0
6762707,1
2438149,1
8170626,1
1968101,0
3429654,0
3766318,0
3772372,0
7552964,1


[4227974,
 1743106,
 6822789,
 378492,
 4674253,
 8219063,
 6574441,
 6486876,
 5430547,
 6373124,
 1810695,
 3087822,
 3265627,
 8876293,
 6959212,
 2350411,
 1020178,
 7482753,
 6130143,
 4271236,
 4611387,
 6592406,
 192792,
 884113,
 1874768,
 97197,
 3198075,
 8651148,
 4115092,
 523142,
 1335246,
 3120351,
 4449065,
 4618141,
 3558175,
 6223866,
 6470683,
 5602431,
 8246861,
 4719316,
 7402150,
 7134630,
 1685767,
 4901499,
 8154112,
 7504059,
 1583109,
 6425537,
 7859197,
 4428096,
 6861590,
 6359355,
 2652303,
 7983145,
 4011002,
 3210637,
 2803844,
 5871180,
 6135107,
 5365090,
 8360220,
 7018000,
 1337718,
 5803812,
 2516361,
 471325,
 6670578,
 7352510,
 8072264,
 5043969,
 1939164,
 2251064,
 5626457,
 8149147,
 5835736,
 5250901,
 8099124,
 3721998,
 4028470,
 1333259,
 7480919,
 3186368,
 1414636,
 4792309,
 5690272,
 1442144,
 4256429,
 8768267,
 3578420,
 7879765,
 8099536,
 7565959,
 6191605,
 6640937,
 6456140,
 4533460,
 7164857,
 5888749,
 1523673,
 1409197,
 455035